# Life Cycle of Process:

### 1. New State
### 2. Runnable State
### 3. Running State
### 4. Blocked/Waiting State
### 5. Termination State

## 1. Process Creation (New State) in Python:
When you create a process object using multiprocessing.Process, Python performs several steps internally to set up the process. This process can be broken down into two main phases: Object Creation and Object Initialization.

### 1.1 Object Creation (__new__ method):
- Before the __init__() method is called, Python uses the __new__() method to allocate memory for the new process object.
- The __new__() method is inherited by the multiprocessing.Process class unless explicitly overridden.
#### At this stage:
- The process object exists in memory, but its attributes (like target, args, etc.) are not yet initialized.
- The object is a blank slate, ready for initialization.
### 1.2 Object Initialization (__init__ method):
- After the object is created, the __init__() method is called to initialize the process object’s attributes.
#### Key attributes that are set during this step:
- target: The function that the process will execute (if provided).
- args: Positional arguments to pass to the target function (default: ()).
- kwargs: Keyword arguments to pass to the target function (default: {}).
- name: The process’s name (default: Process-N, where N is a unique number).
- daemon: Whether the process is a daemon process (inherits from the parent process unless explicitly specified).
- pid: The Process ID is not set during initialization, but it will be created once the process starts.


In [1]:
import multiprocessing
import time

def my_task():
    print("Task is running")

# Custom Process class to track creation and initialization
class CustomProcess(multiprocessing.Process):
    def __new__(cls, *args, **kwargs):
        print("Step 1: Allocating memory for the process object...")
        instance = super().__new__(cls)  # Call parent __new__ to allocate memory
        return instance

    def __init__(self, *args, **kwargs):
        print("Step 2: Initializing the process object...")
        super().__init__(*args, **kwargs)  # Call parent __init__ to set up attributes

# Create a process object
process = CustomProcess(target=my_task)

print("Process object created:", process)
print("Process PID before start:", process.pid)
print("Is process alive after:", process.is_alive())


Step 1: Allocating memory for the process object...
Step 2: Initializing the process object...
Process object created: <CustomProcess name='CustomProcess-1' parent=3365 initial>
Process PID before start: None
Is process alive after: False


## 2. Process State: Runnable and Running State:

### 2.1 Runnable State:
- Runnable State refers to when a process is ready to run, and the operating system's process scheduler can pick it up for execution. At this stage, the process has been started using start(), but it is not yet actively executing. The process may not immediately run because the OS may allocate CPU time to other processes first.

### Life Cycle of Runnable State:

#### i. Process Start:

- The process enters the Runnable state once the start() method is invoked.
- It is now eligible to run, but the OS scheduler may not immediately allocate CPU time to it.

#### ii. Waiting for CPU Time:

- In the Runnable state, the process waits for the operating system to allocate CPU time to it.
-0 Other processes might be running depending on their priority, CPU availability, and the OS's scheduler decisions.

#### iii. OS Scheduling:

- The process remains in the Runnable state while it is waiting for the OS scheduler to pick it up and allocate CPU time.
- The OS scheduler decides which process to run based on factors like priority, process state, and CPU time allocation.

#### iv. Transition to Running State:

- Once the OS scheduler allocates CPU time to the process, it transitions to the Running state, where it begins executing its target function (i.e., the function provided to the target argument when creating the process).
- At this point, the process moves from the Runnable state to the Running state and begins execution on the CPU.

### 2.2 Running State:
- Running State refers to the state where the process is actively executing its code (i.e., the function passed to target). This happens once the OS scheduler has allocated CPU time to the process.
- In the Running state, the process is executing the instructions within the target function (in the run() method or the function provided directly to the target argument).

### Life Cycle of Running State:
#### i. Execution:

- When the process is in the Running state, it is executing its code, consuming CPU resources.
- The process continues to execute until it completes its task or is interrupted by the OS scheduler.

#### ii. Transition to Terminated State:

- Once the process has completed execution, it moves to the Terminated state.
- The exit status of the process is updated, and the process terminates, releasing any resources it was using.


In [2]:
from multiprocessing import Process
import time
import os

def my_task():
    print(f"Process {os.getpid()} is running...")
    time.sleep(2)  # Simulate work for 45 seconds
    print(f"Process {os.getpid()} finished.")

if __name__ == '__main__':
    # Create and start a single process
    process = Process(target=my_task)
    process.start()

    # Print the PID of the process
    print(f"Process {process.pid} started.")

    # Wait for the process to complete
    process.join()

    # Process should have finished now
    print("Process is finished.")


Process 3798 is running...
Process 3798 started.
Process 3798 finished.
Process is finished.


## 3. Blocked/Waiting State:
### 3.1 Blocked State:
- A process enters the Blocked state when it is waiting for a resource to become available or some condition to be satisfied. This often happens when a process needs a resource (such as a lock or I/O resource) that is currently unavailable.

#### When Does a Process Enter Blocked State?
- Blocked State: A process enters the Blocked state when it tries to acquire a resource (like a lock) that is held by another process or is waiting for some resource (e.g., I/O operation).

In [ ]:
# In my case MultiProcessing code is not work fine in jupyter notebook so to see the actual execution run it to another ide.

from multiprocessing import Process, Lock
import time

def process1_task(lock):
    print("Process 1 trying to acquire lock...")
    with lock:  # Trying to acquire lock, will block if already acquired by another process
        print("Process 1 acquired lock.")
        time.sleep(2)
    print("Process 1 released lock.")

def process2_task(lock):
    print("Process 2 acquiring lock...")
    with lock:  # Acquiring lock
        print("Process 2 has the lock, holding it for 2 seconds.")
        time.sleep(2)
    print("Process 2 released lock.")

if __name__ == "__main__":
    lock = Lock()

    process1 = Process(target=process1_task, args=(lock,))
    process2 = Process(target=process2_task, args=(lock,))

    process2.start()  # process2 starts and acquires the lock
    time.sleep(0.5)   # Ensure process2 acquires the lock before process1 starts
    process1.start()  # process1 tries to acquire the lock and gets blocked

    process1.join()
    process2.join()

    print("Both processes are finished.")


Process 2 acquiring lock...
Process 2 has the lock, holding it for 2 seconds.
Process 1 trying to acquire lock...
Process 1 acquired lock.Process 2 released lock.

Process 1 released lock.
Both processes are finished.


### 3.2 Waiting State:
- A process enters the Waiting state when it is explicitly paused and waits for a certain condition, event, or another process to complete. The most common situations for entering the Waiting state are calling join(), sleep(), or wait().

#### When Does a Process Enter Waiting State?
- Waiting State: A process enters the Waiting state when it explicitly pauses its execution, waiting for an event or for another process to finish.

In [ ]:
from multiprocessing import Process
import time

def process1_task():
    print("Process 1 started.")
    time.sleep(3)
    print("Process 1 finished.")

def process2_task():
    print("Process 2 started.")
    time.sleep(1)
    print("Process 2 finished.")

if __name__ == "__main__":
    process1 = Process(target=process1_task)
    process2 = Process(target=process2_task)

    process1.start()
    process2.start()

    process1.join()  # process1 enters the Waiting state until process2 finishes
    process2.join()  # process2 finishes and process1 resumes execution

    print("Both processes are finished.")


Process 2 started.Process 1 started.

Process 2 finished.
Process 1 finished.
Both processes are finished.


## Termination State:
- The process finishes execution, either successfully or with an error, and exits.
- The process is no longer running and cannot be resumed.

#### What Happens During Termination?
- Exit Code: A process sets an exit code (0 for success, non-zero for failure).
- Cleanup: Resources allocated to the process are released.

In [ ]:
from multiprocessing import Process
import time
import os

def normal_process_task():
    time.sleep(2)
    print(f"Process {os.getpid()} completed successfully.")

def error_process_task():
    time.sleep(1)
    raise Exception("Error occurred in process.")

if __name__ == "__main__":
    process1 = Process(target=normal_process_task)
    process2 = Process(target=error_process_task)

    process1.start()
    process2.start()

    process1.join()
    process2.join()

    print(f"Process 1 (PID {process1.pid}) exit code:", process1.exitcode)  # 0 indicates success
    print(f"Process 2 (PID {process2.pid}) exit code:", process2.exitcode)  # Non-zero indicates error


Process Process-15:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-cfb92bed7d2b>", line 11, in error_process_task
    raise Exception("Error occurred in process.")
Exception: Error occurred in process.


Process 3737 completed successfully.
Process 1 (PID 3737) exit code: 0
Process 2 (PID 3738) exit code: 1
